# DPML | Latency Replay

In this notebook, we investigate the reproducibility of transformation sequences captured by `dpml`.

## Load Dependencies

In [1]:
from lineage import LeBatch

from sibyl import *
from datasets import load_dataset

import time
from tqdm.notebook import tqdm

ModuleNotFoundError: No module named 'lineage.logger.text_logger'

## Create Datasets

In [2]:
dataset = load_dataset("glue", "sst2", split="train[:100]")
dataset = dataset.rename_column('sentence', 'text')

Reusing dataset glue (C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


## Replay Test

### Routine to be Tracked

In [3]:
scheduler = SibylTransformScheduler("sentiment")

In [4]:
text, label = dataset['text'], dataset['label'] 
new_text, new_label = [], []

batch_size= 10

scheduler.num_INV = 1
scheduler.num_SIB = 1

records = []
startTime = time.perf_counter()
for i in tqdm(range(0, len(label), batch_size)):
    text_batch = text[i:i+batch_size]
    label_batch = label[i:i+batch_size]
    batch = (text_batch, label_batch)
    for transform in scheduler.sample():
        print(transform)
        # batch = transform.transform_batch(batch)
        batch = LeBatch(batch).apply(transform.transform_batch)
    records.append(batch)
print('Elapsed time: {:6.3f} seconds'.format(time.perf_counter() - startTime))

  0%|          | 0/10 [00:00<?, ?it/s]

Elapsed time: 23.076 seconds


In [14]:
records

[[<LeRecord:
  	 text="envelop new substance from the parental construct  ⚕️",
  	 target="0",
  	 le_attrs={'transformation_provenance': <TransformationProvenance: {(1, "{'class': 'ChangeHypernym', 'num_to_replace': inf, 'return_metadata': True}"), (0, "{'class': 'AddNeutralEmoji', 'num': 1, 'polarity': [-0.05, 0.05], 'return_metadata': True}")}>, 'prev': <LeRecord:
  	 text="hide new secretions from the parental units  ⚕️",
  	 target="0",
  	 le_attrs={'transformation_provenance': <TransformationProvenance: {(0, "{'class': 'AddNeutralEmoji', 'num': 1, 'polarity': [-0.05, 0.05], 'return_metadata': True}")}>, 'prev': <LeRecord:
  	 text="hide new secretions from the parental units ",
  	 target="0",
  	 le_attrs={'transformation_provenance': <TransformationProvenance: set()>, 'feature_provenance': <FeatureProvenance[edit_seq] set()>, 'prev': None}>, 'feature_provenance': <FeatureProvenance[edit_seq] {(0, (7, 8), 'insert: [7,7]-[7,8]')}>, 'granularity': 'word'}>, 'feature_provenance': 

## Replay Functionality

In [66]:
target_record

<LeRecord:
	 text="envelop new substance from the parental construct  ⚕️",
	 target="0",
	 le_attrs={'transformation_provenance': <TransformationProvenance: {(1, "{'class': 'ChangeHypernym', 'num_to_replace': inf, 'return_metadata': True}"), (0, "{'class': 'AddNeutralEmoji', 'num': 1, 'polarity': [-0.05, 0.05], 'return_metadata': True}")}>, 'prev': <LeRecord:
	 text="hide new secretions from the parental units  ⚕️",
	 target="0",
	 le_attrs={'transformation_provenance': <TransformationProvenance: {(0, "{'class': 'AddNeutralEmoji', 'num': 1, 'polarity': [-0.05, 0.05], 'return_metadata': True}")}>, 'prev': <LeRecord:
	 text="hide new secretions from the parental units ",
	 target="0",
	 le_attrs={'transformation_provenance': <TransformationProvenance: set()>, 'feature_provenance': <FeatureProvenance[edit_seq] set()>, 'prev': None}>, 'feature_provenance': <FeatureProvenance[edit_seq] {(0, (7, 8), 'insert: [7,7]-[7,8]')}>, 'granularity': 'word'}>, 'feature_provenance': <FeatureProvenance[e

In [39]:
import ast

In [62]:
def init_transform(transform_history):
    transform_dict = ast.literal_eval(transform_history)
    transform_name = transform_dict.pop("class")
    transform_fn = globals()[transform_name](**transform_dict)
    return transform_fn

In [68]:
target_record = records[0][0]

In [69]:
original_text = text[0]
target_text = target_record.text

In [71]:
tran_history = target_record.le_attrs['transformation_provenance'].history
tran_history = [x[1] for x in sorted(tran_history, key=lambda x: x[0])]

In [73]:
tran_history

["{'class': 'AddNeutralEmoji', 'num': 1, 'polarity': [-0.05, 0.05], 'return_metadata': True}",
 "{'class': 'ChangeHypernym', 'num_to_replace': inf, 'return_metadata': True}"]

In [63]:
tran_fns = [init_transform(h) for h in tran_history]

ValueError: malformed node or string on line 1: <ast.Name object at 0x0000021E6EE92EC0>

In [65]:
transforms[1]

"{'class': 'ChangeHypernym', 'num_to_replace': inf, 'return_metadata': True}"

In [64]:
transform_dict = ast.literal_eval(transforms[1])

ValueError: malformed node or string on line 1: <ast.Name object at 0x0000021E6EE911B0>

In [51]:
tran_name = transform_dict.pop("class")

In [52]:
tran_fn = globals()[tran_name]

In [55]:
tran_fn(**transform_dict)